In [21]:
import os

In [22]:
%pwd

'c:\\Users\\jaide\\OneDrive\\Desktop\\GOPAL\\Projects\\Machine Learning Project\\machine-learning-project'

In [23]:
# os.chdir("../")

In [24]:
%pwd

'c:\\Users\\jaide\\OneDrive\\Desktop\\GOPAL\\Projects\\Machine Learning Project\\machine-learning-project'

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [26]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml , create_directories

In [27]:
class ConfigurationManager:
    def __init__ (
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_data_ingestion_config(self) -> DataIngestionConfig :
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

    
print(f"CONFIG_FILE_PATH: {CONFIG_FILE_PATH.resolve()}")
    

CONFIG_FILE_PATH: C:\Users\jaide\OneDrive\Desktop\GOPAL\Projects\Machine Learning Project\machine-learning-project\config\config.yaml


In [28]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size


In [29]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            print(filename)
            
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
    

    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [30]:
%pwd

'c:\\Users\\jaide\\OneDrive\\Desktop\\GOPAL\\Projects\\Machine Learning Project\\machine-learning-project'

In [31]:
try:
    config=ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-22 16:36:32,271: INFO: common: yaml file: C:\Users\jaide\OneDrive\Desktop\GOPAL\Projects\Machine Learning Project\machine-learning-project\config\config.yaml loaded successfully]
[2025-05-22 16:36:32,276: INFO: common: yaml file: C:\Users\jaide\OneDrive\Desktop\GOPAL\Projects\Machine Learning Project\machine-learning-project\params.yaml loaded successfully]
[2025-05-22 16:36:32,283: INFO: common: yaml file: C:\Users\jaide\OneDrive\Desktop\GOPAL\Projects\Machine Learning Project\machine-learning-project\schema.yaml loaded successfully]
[2025-05-22 16:36:32,287: INFO: common: created directory at: artifacts]
[2025-05-22 16:36:32,290: INFO: common: created directory at: artifacts/data_ingestion]
artifacts/data_ingestion/data.zip
[2025-05-22 16:36:33,214: INFO: 674362924: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 21293
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox